In [11]:
%matplotlib inline
import numpy as np
import pandas as pd
import re
import sklearn as sk
from sklearn.naive_bayes import MultinomialNB
import nltk
import matplotlib.pyplot as plt
# from __future__ import print_function
from __future__ import division
from os import listdir
import re
import collections

In [29]:
def load_train_data(train_data_dir):
    y = []
    X = []
    all_tokens = []
    author_to_id = {}
    for author_id, author in enumerate(listdir(train_data_dir)):
        author_to_id[author] = author_id
        author_path = "%s/%s" % (train_data_dir, author)
        print author, author_id

        for file_name in listdir(author_path):
            full_path = "%s/%s" % (author_path, file_name)
            y.append(author_id)            
            with open(full_path, "r") as f:
                current = canonicalize_words(f.read())
                all_tokens += current
                X.append(np.array(current))
                
#     vocab = Vocabulary(all_tokens)
    vocab = None
    # replace words with ids
#     for i, x in enumerate(X):
#         # X[i] = np.array(x) # This line can be used to make sure your words are useful 
#         X[i] = np.array(vocab.words_to_ids(x))

    return vocab, np.array(X), np.array(y), author_to_id


def id_to_author(author_to_id, id):
    '''
    Takes a dictionary mapping author names to IDs and an ID, and returns
    the author mapped to that ID
    '''
    for author, author_id in author_to_id.iteritems():
        if id == author_id:
            return author
        
def load_eval_data(vocab, eval_data_dir):
    eval_X = {}
    eval_y = {}
    
    for author_id, author in enumerate(listdir(eval_data_dir)):
        author_path = "%s/%s" % (eval_data_dir, author)

        for file_name in listdir(author_path):
            full_path = "%s/%s" % (author_path, file_name)
            
            with open(full_path, "r") as f:
#                 current = vocab.words_to_ids(canonicalize_words(f.read()))
                current = canonicalize_words(f.read())
                
            expanded_X = np.array(current)
            id = file_name.split("_")[2].split(".")[0]
            eval_X[id] = np.array([expanded_X])
            eval_y[id] = np.array([author_to_id[author]])
                
    return eval_X, eval_y

def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word):
    word = word.lower()
    return canonicalize_digits(word) # try to canonicalize numbers

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

def canonicalize_words(words):
    current = []
    rep_dict = {'\n':' '
                ,'\xc2':' '
                ,'\xa0':' '
                ,'\xc2':' '
                ,'\xc3':' '
                ,'\xa9':' '
                ,'\xef':' '
                ,'\xbb':' '
                ,'\xbf':' '
                ,'\xa6':' '
                ,'\xb9':' '
                ,'\xa3':' '
                ,'\xbd':' '
                ,'\xb4':' '
                ,'\xcb':' '
                ,'\x9a':' '
                ,'\x86':' '
                ,'\xcf':' '
                ,'\x84':' '
                ,'\xce':' '
                ,'\x87':' '
                ,'\xe2':' '
                ,'\x80':' '
                ,'\x94':' '
               }
    for word in replace_all(words, rep_dict).split(' '):   
        if word:
            if word[-1] in ('.', ',', '?', ';', '!'):
                punk = word[-1]
                current.append(punk)
                word = word[0:-1]

            word = canonicalize_word(word)
            current.append(word)
    return current


class Vocabulary(object):

  UNK_TOKEN = "<unk>"

  def __init__(self, tokens, size=None):
    self.unigram_counts = collections.Counter(tokens)
    # leave space for "<unk>"
    top_counts = self.unigram_counts.most_common(None if size is None else (size - 1))
    vocab = ([self.UNK_TOKEN] +
             [w for w,c in top_counts])

    # Assign an id to each word, by frequency
    self.id_to_word = dict(enumerate(vocab))
    self.word_to_id = {v:k for k,v in self.id_to_word.iteritems()}
    self.size = len(self.id_to_word)
    if size is not None:
        assert(self.size <= size)

    # For convenience
    self.wordset = set(self.word_to_id.iterkeys())

    # Store special IDs
    self.UNK_ID = self.word_to_id[self.UNK_TOKEN]

  def words_to_ids(self, words):
    return [self.word_to_id.get(w, self.UNK_ID) for w in words]

  def ids_to_words(self, ids):
    return [self.id_to_word[i] for i in ids]

  def ordered_words(self):
    """Return a list of words, ordered by id."""
    return self.ids_to_words(range(self.size))

In [34]:
train_data_dir = './train_final'
vocab, X_train, y_train, author_to_id = load_train_data(train_data_dir)
num_classes = len(np.unique(y_train))

eval_X, eval_y = load_eval_data(vocab, "unknown_data")
test_X, test_y = load_eval_data(vocab, "test_final")

james_madison 0
alexander_hamilton 1


Per Mosteller and Wallace (1963), 12 papers are disputed between Hamilton and Madison (49-58, 62, 63). Jay wrote just 5 (2-5, 64). Three were co-written by Hamilton and Madison (18-20), although the level of contribution from each is disputed. Madison wrote 14 and Hamilton wrote 43. Confusingly, this only adds to 77, which is the number they give for how many federalist papers there are. But there appears to actually be 85. Not sure where this discrepancy comes from. Wikipedia lists the papers as described in Douglass Adair's essay _The Disputed Federalist Papers_, with footnotes indicating which are disputed or joint. https://en.wikipedia.org/wiki/The_Federalist_Papers

From this, we have Madison as the sole author for 14 (10, 14, 37-48) and Hamilton as the sole author for the remaining 51 (1, 6-9, 11-13, 15-17, 21-36, 59-61, 65-85).

For labels, h=Hamilton, m=Madison, j=Jay, hm=Hamilton and Madison, d=disputed.

In [9]:
# j = np.array([2,3,4,5,64])
# m = np.array([10,14,37,38,39,40,41,42,43,44,45,46,47,48])
# hm = np.array([18,19,20])
# d = np.array([49,50,51,52,53,54,55,56,57,58,62,63])

# labels = np.array(['h']*85, dtype=object) # intially label all for Hamilton
# labels[j-1] = 'j'
# labels[m-1] = 'm'
# labels[hm-1] = 'hm'
# labels[d-1] = 'd'

# data['author'] = labels # adding author labels
# le = sk.preprocessing.LabelEncoder()
# le.fit(data['author'])
# data['label'] = le.transform(data['author']) # encoding labels as integers

# # Flagging papers with known authors. Treating dual-authorship as unknown.
# data['known'] = data['author'].apply(lambda author: author in ['j','h','m'])

In [10]:
# # Adair believes that all of the joint and disputed texts were written by Madison, and that Hamilton had little if any input.
# adair_labels = labels
# adair_labels[np.concatenate([hm,d]) - 1] = 'm'

# data['adair'] = adair_labels

For now, training data is all known papers

In [14]:
# train = data[data.known].copy()
# test = data[~data.known].copy()

## Converting data and labels to dataframes for easier handling in code

In [79]:
test_y = [val[0] for val in test_y.values()]
test_X = [val[0] for val in test_X.values()]

In [84]:
X_train

array([ 'having shown that no one of the powers transferred to the federal government is unnecessary or , improper the next question to be considered , is whether the whole mass of them will be dangerous to the portion of authority left in the several . states the adversaries to the plan of the , convention instead of considering in the first place what degree of power was absolutely necessary for the purposes of the federal , government have exhausted themselves in a secondary inquiry into the possible consequences of the proposed degree of power to the governments of the particular . states but if the , union as has been , shown be essential to the security of the people of america against foreign ; danger if it be essential to their security against contentions and wars among the different ; states if it be essential to guard them against those violent and oppressive factions which embitter the blessings of , liberty and against those military establishments which must gradually poi

In [49]:
train = pd.DataFrame(X_train, columns=['body'])
train['label'] = y_train
train['author'] = train['label'].apply(lambda aid: id_to_author(author_to_id, aid))
train['body'] = train['body'].apply(lambda text: ' '.join(text))
train

,body,label,author
0,having shown that no one of the powers transfe...,0,james_madison
1,having reviewed the general form of the propos...,0,james_madison
2,"we have seen the necessity of the , union as o...",0,james_madison
3,the constitution proposed by the convention ma...,0,james_madison
4,it is not a little remarkable that in every ca...,0,james_madison
5,the fourth class comprises the following misce...,0,james_madison
6,"it is sometimes , asked with an air of seeming...",1,alexander_hamilton
7,let us now return to the partition of the judi...,1,alexander_hamilton
8,the residue of the argument against the provis...,1,alexander_hamilton
9,the remaining powers which the plan of the con...,1,alexander_hamilton


In [68]:
test = pd.DataFrame(eval_X.values(), columns=['body'])
test['label'] = eval_y.values()
test['label'] = test['label'].apply(lambda val_list: val_list[0]) # getting rid of list
test['author'] = test['label'].apply(lambda aid: id_to_author(author_to_id, aid))
test['body'] = test['body'].apply(lambda text_series: text_series[0]) # getting rid of nested series
test['body'] = test['body'].apply(lambda text: ' '.join(text))
test

,body,label,author
0,"the united netherlands are a confederacy of , ...",0,james_madison
1,"the remaining charge against the house of , re...",0,james_madison
2,"the author of the ""notes on the state of virgi...",0,james_madison
3,the number of which the house of representativ...,0,james_madison
4,the next view which i shall take of the house ...,0,james_madison
5,the third charge against the house of represen...,0,james_madison
6,the second charge against the house of represe...,0,james_madison
7,"to what , expedient , then shall we finally , ...",0,james_madison
8,"it may be , contended , perhaps that instead o...",0,james_madison
9,"i shall , here , perhaps be reminded of a curr...",0,james_madison


array(['the', 'united', 'netherlands', ..., '.', 'magistracy', 'publius'], 
      dtype='|S19')

# Exploring characteristics of entire corpus

In [50]:
# verifying label counts.
# should be 70 papers, since we have 12 disputed and 3 joint witheld from the full 85
train.groupby('author').label.value_counts()

author              label
alexander_hamilton  1        10
james_madison       0         6
Name: label, dtype: int64

In [51]:
# Counting number of sentences, words, and characters in each paper
train['n_sentences'] = train.body.apply(lambda x: len(nltk.sent_tokenize(x)))
train['n_words'] = train.body.apply(lambda x: len(nltk.word_tokenize(x)))
train['n_characters'] = train.body.apply(lambda x: len(x))

# Grouping counts by author
train.groupby('author')[['n_sentences', 'n_words', 'n_characters']].sum()

,n_sentences,n_words,n_characters
author,,,
alexander_hamilton,654,23481,127135
james_madison,568,19327,105476


In [85]:
# initialize count vectorizer
cv = sk.feature_extraction.text.CountVectorizer(analyzer = "word")

# fitting bag of words model and learning the vocabulary
train_features = cv.fit_transform(train.body)
vocab = cv.get_feature_names()

print('Learned vocab size:', len(vocab))
print('Shape of term-document matrix [n_samples, vocabulary_size]:', train_features.shape)

('Learned vocab size:', 4457)
('Shape of term-document matrix [n_samples, vocabulary_size]:', (16, 4457))


In [106]:
train

,body,label,author,n_sentences,n_words,n_characters
0,having shown that no one of the powers transfe...,0,james_madison,57,2361,13103
1,having reviewed the general form of the propos...,0,james_madison,87,3092,17195
2,"we have seen the necessity of the , union as o...",0,james_madison,57,2359,12880
3,the constitution proposed by the convention ma...,0,james_madison,134,3945,21396
4,it is not a little remarkable that in every ca...,0,james_madison,113,3689,19856
5,the fourth class comprises the following misce...,0,james_madison,120,3881,21046
6,"it is sometimes , asked with an air of seeming...",1,alexander_hamilton,87,2571,14261
7,let us now return to the partition of the judi...,1,alexander_hamilton,124,4382,23257
8,the residue of the argument against the provis...,1,alexander_hamilton,49,1870,9976
9,the remaining powers which the plan of the con...,1,alexander_hamilton,65,2225,12022


In [107]:
train_features

<16x4457 sparse matrix of type '<type 'numpy.int64'>'
	with 11860 stored elements in Compressed Sparse Row format>

In [94]:
authors

array(['james_madison', 'alexander_hamilton'], dtype=object)

In [95]:
train.author

0          james_madison
1          james_madison
2          james_madison
3          james_madison
4          james_madison
5          james_madison
6     alexander_hamilton
7     alexander_hamilton
8     alexander_hamilton
9     alexander_hamilton
10    alexander_hamilton
11    alexander_hamilton
12    alexander_hamilton
13    alexander_hamilton
14    alexander_hamilton
15    alexander_hamilton
Name: author, dtype: object

In [99]:
vocab

[u'10',
 u'11',
 u'1st',
 u'4th',
 u'abandon',
 u'abb',
 u'abetted',
 u'abilities',
 u'ability',
 u'able',
 u'abolish',
 u'abolished',
 u'abolition',
 u'abounding',
 u'about',
 u'above',
 u'abridge',
 u'abridgment',
 u'abroad',
 u'abrogate',
 u'abrogating',
 u'absolute',
 u'absolutely',
 u'absolves',
 u'absorb',
 u'abstract',
 u'absurd',
 u'absurdity',
 u'absurdly',
 u'abundant',
 u'abundantly',
 u'abuse',
 u'abused',
 u'abuses',
 u'acceptance',
 u'accession',
 u'accidental',
 u'accommodate',
 u'accommodation',
 u'accommodations',
 u'accomplice',
 u'accomplish',
 u'accomplished',
 u'accomplishes',
 u'according',
 u'accordingly',
 u'account',
 u'accumulated',
 u'accumulation',
 u'accuracy',
 u'accusation',
 u'accusations',
 u'accused',
 u'accusers',
 u'accustomed',
 u'achaean',
 u'achaeus',
 u'acquaintance',
 u'acquaintances',
 u'acquainted',
 u'acquiesce',
 u'acquire',
 u'acquired',
 u'acquirements',
 u'acquisition',
 u'acquisitions',
 u'acre',
 u'acres',
 u'act',
 u'acting',
 u'action

In [100]:
binarized

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [103]:
sk.preprocessing.label_binarize(train.author, authors)

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [108]:
word_counts

array([[1],
       [1],
       [2],
       ..., 
       [0],
       [2],
       [0]])

In [89]:
# Getting distribution of word counts
authors = train.author.unique()
binarized = sk.preprocessing.label_binarize(train.author, authors) # binarizing for easier summing
word_counts = binarized.T * train_features # matrix multiplication returns counts by word and author
word_counts = word_counts.T # transposing to make columns authors

# creating sorted dataframe with total word counts
count_df = pd.DataFrame(word_counts, columns=authors, index=vocab)
# count_df['total'] = count_df.sum(axis=1) # summing author counts for each word to get total
# count_df.sort_values(by='total', inplace=True, ascending=False)
# count_df

ValueError: Shape of passed values is (1, 4457), indices imply (2, 4457)

In [92]:
word_counts.shape

(4457, 1)

From here we can see that there are obvious stop words that need to be removed. But first, we train a naive bayes with stop words included.

In [113]:
# data['author'] = labels # adding author labels
le = sk.preprocessing.LabelEncoder()
le.fit(train['author'])
# data['label'] = le.transform(data['author']) # encoding labels as integers

LabelEncoder()

In [110]:
test_features = cv.transform(test.body)

clf = MultinomialNB()
clf.fit(train_features, train.label)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
predictions_1 = clf.predict(test_features)

# print(le.inverse_transform(predictions_1))
print predictions_1

[0 0 1 0 1 1 1 0 0 1 0 0 0 0 1 0]


Repeating the above process, but this time using stop words

In [20]:
# initialize count vectorizer
cv = sk.feature_extraction.text.CountVectorizer(analyzer = "word", stop_words='english')

# fitting bag of words model and learning the vocabulary
train_features = cv.fit_transform(train.body)
vocab = cv.get_feature_names()

# Getting distribution of word counts
authors = train.author.unique()
binarized = sk.preprocessing.label_binarize(train.author, authors) # binarizing for easier summing
word_counts = binarized.T * train_features # matrix multiplication returns counts by word and author
word_counts = word_counts.T # transposing to make columns authors

# creating sorted dataframe with total word counts
count_df = pd.DataFrame(word_counts, columns=authors, index=vocab)
count_df['total'] = count_df.sum(axis=1) # summing author counts for each word to get total
count_df.sort_values(by='total', inplace=True, ascending=False)
count_df

,h,j,m,total
states,459,19,245,723
government,419,45,227,691
state,435,15,155,605
power,400,19,138,557
constitution,240,6,143,389
people,221,43,122,386
union,242,22,75,339
national,244,31,53,328
authority,179,1,75,255
great,167,15,70,252


In [111]:
# initialize count vectorizer
cv = sk.feature_extraction.text.CountVectorizer(analyzer = "word", stop_words='english')

# fitting bag of words model and learning the vocabulary
train_features = cv.fit_transform(train.body)
vocab = cv.get_feature_names()

test_features = cv.transform(test.body)

clf = MultinomialNB()
clf.fit(train_features, train.label)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
predictions_2 = clf.predict(test_features)

# print(le.inverse_transform(predictions_2))
print predictions_2

[0 0 0 0 1 1 1 0 0 1 1 0 0 0 1 0]


In [114]:
print(le.inverse_transform(predictions_2))

['alexander_hamilton' 'alexander_hamilton' 'alexander_hamilton'
 'alexander_hamilton' 'james_madison' 'james_madison' 'james_madison'
 'alexander_hamilton' 'alexander_hamilton' 'james_madison' 'james_madison'
 'alexander_hamilton' 'alexander_hamilton' 'alexander_hamilton'
 'james_madison' 'alexander_hamilton']


In [118]:
le.inverse_transform([0])

array(['alexander_hamilton'], dtype=object)

Now trying with tf-idf transform and stop words

In [112]:
# initialize count vectorizer
cv = sk.feature_extraction.text.TfidfVectorizer(analyzer = "word", stop_words='english')

# fitting bag of words model and learning the vocabulary
train_features = cv.fit_transform(train.body)
vocab = cv.get_feature_names()

# Getting distribution of word counts
authors = train.author.unique()
binarized = sk.preprocessing.label_binarize(train.author, authors) # binarizing for easier summing
word_counts = binarized.T * train_features # matrix multiplication returns counts by word and author
word_counts = word_counts.T # transposing to make columns authors

test_features = cv.transform(test.body)

clf = MultinomialNB()
clf.fit(train_features, train.label)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
predictions_3 = clf.predict(test_features)

# print(le.inverse_transform(predictions_3))
print predictions_3

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


## Some results

In [23]:
results = pd.DataFrame([le.inverse_transform(predictions_1),
                        le.inverse_transform(predictions_2),
                        le.inverse_transform(predictions_3)])
results = results.transpose()
results.columns = ['1', '2', '3']
results.set_index(test.index, inplace=True)
results

,1,2,3
num,,,
18.0,m,m,h
19.0,h,h,h
20.0,h,h,h
49.0,h,h,h
50.0,m,m,h
51.0,m,m,h
52.0,h,h,h
53.0,h,h,h
54.0,h,h,h


Recall that Adair believes all of the disputed works, including the three joint-authorship works (18-20), were written by Madison. Mosteller and Wallace's analysis also leaned heavily toward Madison with the exception of papers 55 and 20, where the evidence was less-heavily toward Madison or ambiguous, respectively.

Unfortunately, these basic bag-of-words classifiers with Naive Bayes predict predominantly Hamilton. Clearly we have some work to do.